In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/netflix-shows/netflix_titles.csv")
df_a = pd.DataFrame(index= df.index)
df

### Analysis Columns

In [ ]:
def get_values(origin, value_list, symbol=','):
    if not origin or type(origin) is float:
        return 0
    
    values = origin.strip().split(symbol)
        
    result = 0
    
    for value in values:
        value = value.strip()
        if value in value_list:
            idx = value_list.index(value) + 1
        else:
            value_list.append(value)
            idx = len(value_list)
        if idx > 0:
             result |= 2 ** idx
                
    return result
    
   
all_types = []
df_a['type'] = df['type'].apply(get_values, value_list=all_types)

df['director'] = df['director'].fillna('')
all_directors = []
df_a['director'] = df['director'].apply(get_values, value_list=all_directors)

all_coutries = []
df_a['country'] = df['country'].apply(get_values, value_list=all_coutries)

all_rating = {
    'TV-Y': 0,
    'TV-PG': 7,
    'TV-Y7': 7,
    'TV-Y7-FV': 7,
    'TV-14': 16,
    'TV-MA': 17,    
    'G': 0,
    'R': 18,
    'NR': 18,
    'PG': 7,
    'PG-13': 13,
    'TV-G': 7,    
    'UR': 18,
    'NC-17': 18
}

df['rating'] = df['rating'].fillna('NR')
df_a['rating'] = df['rating'].apply(lambda x: all_rating[x])

all_listed_in = []
df_a['listed_in'] = df['listed_in'].apply(get_values, value_list=all_listed_in)

# date_added 在netflix上架的年份

# release_year 實際發布年份
df_a['release_year'] = df['release_year']

# director's movie count
# df_a.groupby(['director']).size().sort_values(ascending=False)
df_a

In [ ]:
# 年份增長
from matplotlib.pyplot import MultipleLocator
x_major_locator=MultipleLocator(10)

ax = df_a.groupby(['release_year']).size().plot(kind='bar', figsize=(20, 6))


In [ ]:
# 電影年份增長
df_a[df_a['type'] & 2 == 2].groupby(['release_year']).size().plot(kind='bar', figsize=(20, 6))


In [ ]:
# 電視年份增長
df_a[df_a['type'] & 4 == 4].groupby(['release_year']).size().plot(kind='bar', figsize=(20, 6))

# 

In [ ]:
# 國家出片占比
import math

coutry_group = df_a.groupby(['country']).size()
coutry_group = coutry_group.reset_index().rename({ 0: 'count'}, axis='columns')
coutry_group['country'] = coutry_group.apply(lambda x:  all_coutries[int(math.log2(x['country']))-1] if x['country'] > 0 else 'unknown', axis=1)
coutry_group[coutry_group['count'] > 100].set_index('country').plot(kind='pie', y='count',autopct='%1.1f%%', figsize=(20, 10))

In [ ]:
# 預期